### 네이버 영화평 감성 분석

In [1]:
!pip install konlpy > /dev/null

In [2]:
import numpy as np
import pandas as pd
import warnings 
warnings.filterwarnings('ignore')

In [5]:
from google.colab import files
up = files.upload()

Saving naver_movie_test_전처리완료.tsv to naver_movie_test_전처리완료.tsv
Saving naver_movie_train_전처리완료.tsv to naver_movie_train_전처리완료.tsv


In [7]:
# 올린 파일 이름 반환해줌
up.keys()[1]

dict_keys(['naver_movie_test_전처리완료.tsv', 'naver_movie_train_전처리완료.tsv'])

In [11]:
# dict형태라 리스트로 넣어야함
train_df = pd.read_csv(list(up.keys())[1], sep='\t')
test_df = pd.read_csv(list(up.keys())[0], sep='\t')

In [12]:
from konlpy.tag import Okt
okt = Okt()

In [14]:
stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다','을','ㅋㅋ','ㅠㅠ','ㅎㅎ']


In [17]:
def okt_tokenizer(text):
    
    morphs = okt.morphs(text, stem=True)
    tokens = [word for word in morphs if word not in stopwords]
    return tokens

In [18]:
okt_tokenizer('열심히 일한 당신 주말엔 여행을 떠나봐요')

['열심히', '일', '당신', '주말', '엔', '여행', '떠나다', '보다']

- Pipeline으로 피쳐 변환과 분류를 동시에

In [22]:
train_df.head()


,id,document,label
0,9976970,아 더빙 진짜 짜증나네요 목소리,0
1,3819312,흠 포스터보고 초딩영화줄 오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 솔직히 재미는 없다 평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화 스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


In [19]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression


In [23]:
pipeline = Pipeline([
    ('TFIDF', TfidfVectorizer(tokenizer=okt_tokenizer)),
    ('LR', LogisticRegression(random_state=2022))
])
# 토크나이저를 통해서 알아서 학습함
# 함수 지정한 tokenizer=okt_tokenizer으로 사용.
pipeline.fit(train_df.document, train_df.label)

Pipeline(steps=[('TFIDF',
                 TfidfVectorizer(tokenizer=<function okt_tokenizer at 0x7f3e80800710>)),
                ('LR', LogisticRegression(random_state=2022))])

In [25]:
pipeline.score(test_df.document, test_df.label)

0.8425038892982887

- 실제 데이터 테스트

In [28]:
review1 = ['모든 국민이 봤으면 하는 영화입니다', 
            '생각보다 지루하고 별로였네요... 보면서 좀 졸았습니다... ㅜ']
import re
review1 = map(lambda x: re.sub('[^가-힣]', ' ', x), review1)
pipeline.predict(review1)

array([1, 0])

- 최적 파라메터 찾기
    - 매 시행마다 한글 형태소 분석을 하느라 시간이 너무 오래 걸림
    - 최적 파라메터를 찾으려고 하면 한글 형태소 분석을 한 데이터로 할것

In [32]:
from sklearn.model_selection import GridSearchCV
params = {
    'TFIDF__ngram_range' : [(1,2)],
    'TFIDF__max_df' : [0.95],
    'LR__C' : [1]
}
grid_pipe = GridSearchCV(
    pipeline, params, scoring='accuracy', cv=3
)
%time grid_pipe.fit(train_df.document, train_df.label)
print(grid_pipe.best_params_, grid_pipe.best_score_)
best_pipe = grid_pipe.best_estimator_
best_pipe.score(test_df.document, test_df.label)

CPU times: user 31min 53s, sys: 56 s, total: 32min 49s
Wall time: 31min 31s
{'LR__C': 1, 'TFIDF__max_df': 0.95, 'TFIDF__ngram_range': (1, 2)} 0.8514577460921422


0.8583067223450421